In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from datetime import date, timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from collections import Counter
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from datetime import datetime
import selenium.common.exceptions
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

import os 
import time
import logging
import pandas as pd

In [3]:
def get_weather_data_for_dates8(start_year, start_month, start_day, end_year, end_month, end_day, browser):
    weather_data = {}
    current_date = datetime(start_year, start_month, start_day)
    start_date = current_date
    end_date = datetime(end_year, end_month, end_day)
    problematic_urls = []
    
    # Load the csv file into a dataframe
    df = pd.read_csv("countries_for_search.csv")
    
    # Extract the location names and serialized hemisphere values into separate lists
    cities = df["Location"].tolist()
    print(cities)
    serialized_hemispheres = df["Serialized Hemisphere"].tolist()
    
     # Set the save directory and file name
   # save_dir = "weather_data"
   # os.makedirs(save_dir, exist_ok=True)
   # file_name = f"{save_dir}/weather_data_{start_year}-{start_month}-{start_day}
   # _to_{end_year}-{end_month}-{end_day}.csv"
    
    # Check if the file already exists and load the existing data
    #if os.path.isfile(file_name):
     #   existing_data = pd.read_csv(file_name, index_col=0)
    #else:
     #   existing_data = pd.DataFrame()
        
    # Loop through each city in the list
    for i, city in enumerate(cities):
        while current_date <= end_date:
            try:
                year, month, day = current_date.year, current_date.month, current_date.day
                if not browser.current_window_handle: # Check if browser is still open
                    browser.quit()
                    time.sleep(15)
                    browser = webdriver.Firefox()
                    browser.maximize_window()
                date_data = get_weather_data4(year, month, day, city, browser)
                
                # Add the city and serialized hemisphere data to the date_data dictionary
                date_data["Location"] = city
                date_data["Serialized Hemisphere"] = serialized_hemispheres[i]
               
                # Add the date and location as a tuple key to the weather_data dictionary
                weather_data[current_date.strftime("%Y-%m-%d"),city] = date_data
            
                """# Check if it's been one year since the last time the data was saved
                if len(existing_data) > 0:
                    last_saved_date = pd.to_datetime(existing_data.index[-1][0])
                    if current_date - last_saved_date >= timedelta(days=365):
                         # Combine the new and existing data and save it to the CSV file
                        all_data = pd.concat([existing_data, pd.DataFrame.from_dict(weather_data, orient="index")])
                        all_data.to_csv(file_name)
                        weather_data = {}"""
            
            
            
            except Exception as e:
                max_retries = 1 # usually on - 5 tries 
                retries = 0
                while retries < max_retries:
                    try:
                        browser.quit()
                        time.sleep(15)
                        browser = webdriver.Firefox()
                        browser.maximize_window()
                        date_data = get_weather_data4(year, month, day, city, browser)
                        weather_data[current_date.strftime("%Y-%m-%d"),city] = date_data
                        break
                    except Exception as e:
                        retries += 1
                        time.sleep(30)
                        logging.error("An error occurred while getting data for date %s: %s", current_date, e)
                    except SessionNotFoundException:
                        browser.quit()
                        time.sleep(15)
                        browser = webdriver.Firefox()
                        browser.maximize_window()
                        date_data = get_weather_data4(year, month, day, city, browser)
                        weather_data[current_date.strftime("%Y-%m-%d"),city] = date_data
                    if retries == max_retries:
                        current_url = browser.current_url
                        browser.quit()
                        logging.error("Maximum number of retries reached for date %s", current_date)
                        #problematic_urls.append((f"{current_date.strftime("%Y-%m-%d")}, {city}", current_url))
                        problematic_urls.append((f"{current_date.strftime('%Y-%m-%d')}, {city}", current_url))
            current_date += timedelta(days=1)
        current_date = start_date
    browser.quit()
    return weather_data, problematic_urls

In [4]:
def get_weather_data4( year, month, day, city, browser):
    try:
        wait = WebDriverWait(browser, 20)
        browser.get('https://www.wunderground.com/history') 
        wait.until(EC.presence_of_element_located((By.ID, 'yearSelection')))
        wait.until(EC.presence_of_element_located((By.ID, 'historySearch')))
        
        # Select year, month, and day from the drop-down menus
        actions = ActionChains(browser)
        year_select = Select(browser.find_element(By.ID,'yearSelection'))
        year_select.select_by_value(str((2023-year))+':'+' '+str(year))
        month_select = Select(browser.find_element(By.ID,'monthSelection'))
        month_select.select_by_value(str(month))
        day_select = Select(browser.find_element(By.ID,'daySelection'))
        day_select.select_by_value(str((day-1))+':'+' '+str(day))
        # Waiting for choosing destination 
        WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input#historySearch[name='historySearch']"))).send_keys(city)
        WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, f"//search-autocomplete//ul[@class='ui-autocomplete ui-front ui-menu ui-widget ui-widget-content ui-corner-all']//li/a/span/span[text()='{city}']"))).click()
        WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input#dateSubmit[value='View']"))).click()
        
        # Wait for the data to load
        wait.until_not(EC.presence_of_element_located((By.XPATH, '//div[@class="loading-indicator"]')))
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, 'row')))
        wait.until(EC.presence_of_element_located((By.XPATH, '//table[@class="mat-table cdk-table mat-sort ng-star-inserted"]')))
    
    except TimeoutError:
        raise Exception("Timeout Error: Could not load data from URL")
    
    # Extract the data from the table using BeautifulSoup
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    table = soup.select_one('table[class*="mat-table cdk-table mat-sort ng-star-inserted"]')
    rows = table.find_all('tr',{'class':'mat-row cdk-row ng-star-inserted'})
  
    time = []
    temperature = []
    dew_point = []
    humidity = []
    precipitation = []
    wind_speed = []
    wind_dir = []
    condition = []
    
    for row in rows[1:len(rows)-1]:
        cells = row.find_all('td')
       
        time.append(cells[0].text)
        
        temperature_element = cells[1].find('span', class_='wu-value wu-value-to')
        temperature.append(temperature_element.text)
        
        dew_point_element = cells[2].find('span', class_='wu-value wu-value-to')
        dew_point.append(dew_point_element.text)
        
        humidity_element = cells[3].find('span', class_='wu-value wu-value-to')
        humidity.append(humidity_element.text)
        
        wind_dir_element = cells[4].find('span', class_='ng-star-inserted')
        wind_dir.append(wind_dir_element.text)
        
        wind_speed_element = cells[5].find('span', class_='wu-value wu-value-to')
        wind_speed.append(wind_speed_element.text)
        
        precipitation_element = cells[8].find('span', class_='wu-value wu-value-to')
        precipitation.append(precipitation_element.text)
       
        condition_element = cells[9].find('span', class_='ng-star-inserted')
        condition.append(condition_element.text)
        
    #calculating avg
    temperature_avg = fahrenheit_to_celsius(sum(float(t) for t in temperature) / len(temperature))
    precipitation_avg = sum(float(p) for p in precipitation) / len(precipitation)
    wind_avg = mph_to_kmh(sum(float(w) for w in wind_speed) / len(wind_speed))
    humidity_avg = sum(float(r) for r in humidity) / len(humidity)
    dew_point_avg = fahrenheit_to_celsius(sum(float(r) for r in dew_point) / len(dew_point))
    wind_dir_mf = most_frequent_wind_direction(wind_dir)    
    condition_mf = most_frequent_string(condition)
    
    data = {'Year': year, 'Month': month, 'Day': day, 
            'Temperature': temperature_avg, 
            'Temp Dew Point': dew_point_avg,
            'Humidity': humidity_avg,
            'Wind Direction': wind_dir_mf, 
            'Wind Speed': wind_avg, 
            'Precipitation': precipitation_avg,
            'Condition': condition_mf}
    return data

In [5]:
def most_frequent_wind_direction(arr):
    wind_directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE',
                       'SE', 'SSE', 'S', 'SSW', 'SW', 'WSW',
                       'W', 'WNW', 'NW', 'NNW', 'CALM','VAR']
    
    valid_directions = []
    for direction in arr:
        if direction in wind_directions:
            valid_directions.append(direction)
        #else:
            #print(f"Warning: Invalid wind direction '{direction}' found in the input array")
    
    count = Counter(valid_directions)
    return count.most_common(1)[0][0]

def most_frequent_string(strings):
    frequency = {}
    max_frequency = 0
    most_frequent_string = None
    for string in strings:
        if string in frequency:
            frequency[string] += 1
        else:
            frequency[string] = 1
        if frequency[string] > max_frequency:
            max_frequency = frequency[string]
            most_frequent_string = string
    return most_frequent_string
 
    # Conversions
def fahrenheit_to_celsius(temp_fahrenheit):
    temp_celsius = (temp_fahrenheit - 32) * 5/9
    return temp_celsius

def mph_to_kmh(mph):
    kmh = mph * 1.609344
    return kmh

In [6]:
#MAIN
browser = webdriver.Firefox()
browser.maximize_window()
weather_data, problematic_urls = get_weather_data_for_dates8(2015, 1, 1, 2023, 2, 15, browser) #17/09/2010 - problematic
weather_data_df = pd.DataFrame.from_dict(weather_data).transpose()
weather_data_df = weather_data_df.astype({'Year': int, 'Month': int, 'Day': int})

#saving collected data:
weather_data_df.to_csv("weather_data.csv", index=False)
weather_data_df

['Indira Gandhi International Airport, NCT of Delhi, India (VIDP)', 'Port Elizabeth Airport, Eastern Cape, South Africa (FAPE)']


ERROR:root:An error occurred while getting data for date 2018-04-29 00:00:00: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

ERROR:root:Maximum number of retries reached for date 2018-04-29 00:00:00
ERROR:root:An error occurred while getting data for date 2019-06-10 00:00:00: division by zero
ERROR:root:Maximum number of retries reached for date 2019-06-10 00:00:00
ERROR:root:An error occurred while getting data for date 2020-08-03 00:00:00: division by zero
ERROR:root:Maximum number of retries reached for date 2020-08-03 00:00:00
ERROR:root:An error occurred while getting data for date 2020-08-04 00:00:00: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrom

ERROR:root:Maximum number of retries reached for date 2020-08-04 00:00:00
ERROR:root:An error occurred while getting data for date 2022-07-03 00:00:00: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:180:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:392:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:275:16

ERROR:root:Maximum number of retries reached for date 2022-07-03 00:00:00
ERROR:root:An error occurred while getting data for date 2015-04-11 00:00:00: division by zero
ERROR:root:Maximum number of retries reached for date 2015-04-11 00:00:00
ERROR:root:An error occurred while getting data for date 2015-12-06 00:00:00: list index out of range
ERROR:root:Maximum number of retries reached for date 2015-12-06 00:00:00


ERROR:root:An error occurred while getting data for date 2015-12-07 00:00:00: list index out of range
ERROR:root:Maximum number of retries reached for date 2015-12-07 00:00:00
ERROR:root:An error occurred while getting data for date 2015-12-08 00:00:00: list index out of range
ERROR:root:Maximum number of retries reached for date 2015-12-08 00:00:00
ERROR:root:An error occurred while getting data for date 2015-12-09 00:00:00: list index out of range
ERROR:root:Maximum number of retries reached for date 2015-12-09 00:00:00


ERROR:root:An error occurred while getting data for date 2018-07-26 00:00:00: list index out of range
ERROR:root:Maximum number of retries reached for date 2018-07-26 00:00:00
ERROR:root:An error occurred while getting data for date 2018-08-28 00:00:00: division by zero
ERROR:root:Maximum number of retries reached for date 2018-08-28 00:00:00
ERROR:root:An error occurred while getting data for date 2018-10-04 00:00:00: list index out of range
ERROR:root:Maximum number of retries reached for date 2018-10-04 00:00:00


ERROR:root:An error occurred while getting data for date 2021-02-20 00:00:00: division by zero
ERROR:root:Maximum number of retries reached for date 2021-02-20 00:00:00


Year  Month  \
2015-01-01 Indira Gandhi International Airport, NCT of Del...  2015      1   
2015-01-02 Indira Gandhi International Airport, NCT of Del...  2015      1   
2015-01-03 Indira Gandhi International Airport, NCT of Del...  2015      1   
2015-01-04 Indira Gandhi International Airport, NCT of Del...  2015      1   
2015-01-05 Indira Gandhi International Airport, NCT of Del...  2015      1   
...                                                             ...    ...   
2023-02-11 Port Elizabeth Airport, Eastern Cape, South Afr...  2023      2   
2023-02-12 Port Elizabeth Airport, Eastern Cape, South Afr...  2023      2   
2023-02-13 Port Elizabeth Airport, Eastern Cape, South Afr...  2023      2   
2023-02-14 Port Elizabeth Airport, Eastern Cape, South Afr...  2023      2   
2023-02-15 Port Elizabeth Airport, Eastern Cape, South Afr...  2023      2   

                                                               Day  \
2015-01-01 Indira Gandhi International Airport, NCT of Del...    1   
2015-01-02 Indira Gandhi International Airport, NCT of Del...    2   
2015-01-03 Indira Gandhi International Airport, NCT of Del...    3   
2015-01-04 Indira Gandhi International Airport, NCT of Del...    4   
2015-01-05 Indira Gandhi International Airport, NCT of Del...    5   
...                                                            ...   
2023-02-11 Port Elizabeth Airport, Eastern Cape, South Afr...   11   
2023-02-12 Port Elizabeth Airport, Eastern Cape, South Afr...   12   
2023-02-13 Port Elizabeth Airport, Eastern Cape, South Afr...   13   
2023-02-14 Port Elizabeth Airport, Eastern Cape, South Afr...   14   
2023-02-15 Port Elizabeth Airport, Eastern Cape, South Afr...   15   

                                                              Temperature  \
2015-01-01 Indira Gandhi International Airport, NCT of Del...   13.958333   
2015-01-02 Indira Gandhi International Airport, NCT of Del...   13.994709   
2015-01-03 Indira Gandhi International Airport, NCT of Del...   13.111111   
2015-01-04 Indira Gandhi International Airport, NCT of Del...   13.505291   
2015-01-05 Indira Gandhi International Airport, NCT of Del...   12.674419   
...                                                                   ...   
2023-02-11 Port Elizabeth Airport, Eastern Cape, South Afr...   23.217593   
2023-02-12 Port Elizabeth Airport, Eastern Cape, South Afr...   23.315972   
2023-02-13 Port Elizabeth Airport, Eastern Cape, South Afr...   23.888889   
2023-02-14 Port Elizabeth Airport, Eastern Cape, South Afr...   23.496732   
2023-02-15 Port Elizabeth Airport, Eastern Cape, South Afr...   22.954545   

                                                              Temp Dew Point  \
2015-01-01 Indira Gandhi International Airport, NCT of Del...      10.736111   
2015-01-02 Indira Gandhi International Airport, NCT of Del...      12.407407   
2015-01-03 Indira Gandhi International Airport, NCT of Del...      11.569444   
2015-01-04 Indira Gandhi International Airport, NCT of Del...      10.806878   
2015-01-05 Indira Gandhi International Airport, NCT of Del...       7.997416   
...                                                                      ...   
2023-02-11 Port Elizabeth Airport, Eastern Cape, South Afr...      18.981481   
2023-02-12 Port Elizabeth Airport, Eastern Cape, South Afr...      19.809028   
2023-02-13 Port Elizabeth Airport, Eastern Cape, South Afr...      19.876543   
2023-02-14 Port Elizabeth Airport, Eastern Cape, South Afr...      21.013072   
2023-02-15 Port Elizabeth Airport, Eastern Cape, South Afr...      19.090909   

                                                                Humidity  \
2015-01-01 Indira Gandhi International Airport, NCT of Del...     83.325   
2015-01-02 Indira Gandhi International Airport, NCT of Del...  90.190476   
2015-01-03 Indira Gandhi International Airport, NCT of Del...      85.95   
2015-01-04 Indira Gandhi International Airport, NCT of Del...  85.285714   
2015-01-05 Indira G

In [7]:
# Create a DataFrame from the problematic_urls list
problematic_urls_df = pd.DataFrame(problematic_urls, columns=['Date', 'URL'])

# Save the DataFrame to a CSV file
problematic_urls_df.to_csv('problematic_urls_for_weather_data.csv', index=False)
problematic_urls

[('2018-04-29, Indira Gandhi International Airport, NCT of Delhi, India (VIDP)',
  'https://www.wunderground.com/history/daily/VIDP/date/2018-4-29'),
 ('2019-06-10, Indira Gandhi International Airport, NCT of Delhi, India (VIDP)',
  'https://www.wunderground.com/history/daily/VIDP/date/2019-6-10'),
 ('2020-08-03, Indira Gandhi International Airport, NCT of Delhi, India (VIDP)',
  'https://www.wunderground.com/history/daily/VIDP/date/2020-8-3'),
 ('2020-08-04, Indira Gandhi International Airport, NCT of Delhi, India (VIDP)',
  'https://www.wunderground.com/history/daily/VIDP/date/2020-8-4'),
 ('2022-07-03, Indira Gandhi International Airport, NCT of Delhi, India (VIDP)',
  'https://www.wunderground.com/history/daily/VIDP/date/2022-7-3'),
 ('2015-04-11, Port Elizabeth Airport, Eastern Cape, South Africa (FAPE)',
  'https://www.wunderground.com/history/daily/FAPE/date/2015-4-11'),
 ('2015-12-06, Port Elizabeth Airport, Eastern Cape, South Africa (FAPE)',
  'https://www.wunderground.com/hi

In [8]:
weather_data_df.to_csv("weather_data-index-True.csv", index=True)